In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Accident Hotspots Analysis") \
    .getOrCreate()

df = spark.read.csv('Road.csv', header=True, inferSchema=True)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/09 01:05:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Cleaning Time

In [2]:
# Converting String to TimeStamp

from pyspark.sql.functions import to_timestamp

df = df.withColumn('Time', to_timestamp(df['Time'], 'yyyy-MM-dd HH:mm:ss'))

In [3]:
# Extract Hour and Minutes From Time Field

from pyspark.sql.functions import hour, minute

df = df.withColumn('Hour', hour(df['Time']))
df = df.withColumn('Minute', minute(df['Time']))

In [4]:
# Handling invalid/null Quantities from Hour and Minute

from pyspark.sql.functions import hour, minute

cols_to_keep = ['Time','Day_of_week','Area_accident_occured','Number_of_casualties','Accident_severity']
df = df.select(*cols_to_keep)
df = df.filter(df['Time'].isNotNull())

df = df.withColumn('Hour', hour(df['Time']))
df = df.withColumn('Minute', minute(df['Time']))

In [5]:
df.show()

+-------------------+-----------+---------------------+--------------------+-----------------+----+------+
|               Time|Day_of_week|Area_accident_occured|Number_of_casualties|Accident_severity|Hour|Minute|
+-------------------+-----------+---------------------+--------------------+-----------------+----+------+
|2023-12-09 17:02:00|     Monday|    Residential areas|                   2|    Slight Injury|  17|     2|
|2023-12-09 17:02:00|     Monday|         Office areas|                   2|    Slight Injury|  17|     2|
|2023-12-09 17:02:00|     Monday|   Recreational areas|                   2|   Serious Injury|  17|     2|
|2023-12-09 01:06:00|     Sunday|         Office areas|                   2|    Slight Injury|   1|     6|
|2023-12-09 01:06:00|     Sunday|     Industrial areas|                   2|    Slight Injury|   1|     6|
|2023-12-09 14:15:00|     Friday|                 NULL|                   1|    Slight Injury|  14|    15|
|2023-12-09 17:30:00|  Wednesday|    

# Cleaning Day_of_week

In [8]:
# Converting Day_of_week to numberical format

from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType

# Assuming df is your DataFrame

# Create a user-defined function for converting days to numbers
days_dict = {'Monday': 1, 'Tuesday': 2, 'Wednesday': 3, 'Thursday': 4, 'Friday': 5, 'Saturday': 6, 'Sunday': 7}
convert_day_udf = udf(lambda day: days_dict.get(day, 0), IntegerType())

# Apply the UDF to the 'Day_of_week' column
df = df.withColumn('Day_of_week_numeric', convert_day_udf(df['Day_of_week']))

In [9]:
df.show()

+-------------------+-----------+---------------------+--------------------+-----------------+----+------+-------------------+
|               Time|Day_of_week|Area_accident_occured|Number_of_casualties|Accident_severity|Hour|Minute|Day_of_week_numeric|
+-------------------+-----------+---------------------+--------------------+-----------------+----+------+-------------------+
|2023-12-08 17:02:00|     Monday|    Residential areas|                   2|    Slight Injury|  17|     2|                  1|
|2023-12-08 17:02:00|     Monday|         Office areas|                   2|    Slight Injury|  17|     2|                  1|
|2023-12-08 17:02:00|     Monday|   Recreational areas|                   2|   Serious Injury|  17|     2|                  1|
|2023-12-08 01:06:00|     Sunday|         Office areas|                   2|    Slight Injury|   1|     6|                  7|
|2023-12-08 01:06:00|     Sunday|     Industrial areas|                   2|    Slight Injury|   1|     6|     

# Cleaning Age_band_of_driver

In [5]:
from sklearn.preprocessing import OneHotEncoder
import pandas as pd

# Selecting the 'Age_band_of_driver' column
age_band = df['Age_band_of_driver']

# Creating the OneHotEncoder object
encoder = OneHotEncoder(sparse=False)

# Applying one-hot encoding
age_band_encoded = encoder.fit_transform(age_band)

# Converting the encoded data into a DataFrame
age_band_encoded_df = pd.DataFrame(age_band_encoded, columns=encoder.get_feature_names_out(['Age_band_of_driver']))

# Displaying the first few rows of the encoded DataFrame
age_band_encoded_df.head()

/usr/local/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


TypeError: len() of unsized object

# Cleaning Light_conditions

In [29]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.builder.appName("example").getOrCreate()

# Read in the data  
df = spark.read.csv('Road.csv', header=True, inferSchema=True)

# Filter out rows with null or 'Unknown' values in 'Weather_conditions'
df_filtered = df.filter((df.Weather_conditions.isNotNull()) & (df.Weather_conditions != 'Unknown'))

# StringIndexer to convert the 'Light_conditions' column to numeric indices
stringIndexer = StringIndexer(inputCol="Light_conditions", outputCol="Light_conditions_Index")

# Fit the indexer to the data and transform it
indexed = stringIndexer.fit(df).transform(df_filtered)

# OneHotEncoder to convert indexed numbers to one-hot encoded values
encoder = OneHotEncoder(inputCols=["Light_conditions_Index"], outputCols=["Light_conditions_Encoded"])

# Fit and transform
encoded = encoder.fit(indexed).transform(indexed)

# Show the results
encoded.select("Light_conditions", "Light_conditions_Index", "Light_conditions_Encoded").show()


+--------------------+----------------------+------------------------+
|    Light_conditions|Light_conditions_Index|Light_conditions_Encoded|
+--------------------+----------------------+------------------------+
|            Daylight|                   0.0|           (3,[0],[1.0])|
|            Daylight|                   0.0|           (3,[0],[1.0])|
|            Daylight|                   0.0|           (3,[0],[1.0])|
|Darkness - lights...|                   1.0|           (3,[1],[1.0])|
|Darkness - lights...|                   1.0|           (3,[1],[1.0])|
|            Daylight|                   0.0|           (3,[0],[1.0])|
|            Daylight|                   0.0|           (3,[0],[1.0])|
|            Daylight|                   0.0|           (3,[0],[1.0])|
|            Daylight|                   0.0|           (3,[0],[1.0])|
|            Daylight|                   0.0|           (3,[0],[1.0])|
|            Daylight|                   0.0|           (3,[0],[1.0])|
|     

# Cleaning Weather_conditions

In [30]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer, OneHotEncoder

# Initialize Spark session
spark = SparkSession.builder.appName("DataCleaning").getOrCreate()

# Read in the data  from csv file
df = spark.read.csv('Road.csv', header=True, inferSchema=True)

# Filter out rows with null or 'Unknown' values in 'Weather_conditions'
df_filtered = df.filter((df.Weather_conditions.isNotNull()) & (df.Weather_conditions != 'Unknown'))

# StringIndexer to convert the 'Weather_conditions' column to numeric indices
stringIndexer = StringIndexer(inputCol="Weather_conditions", outputCol="Weather_conditions_Index")

# Fit the indexer to the data and transform it
indexed = stringIndexer.fit(df).transform(df_filtered)

# OneHotEncoder to convert indexed numbers to one-hot encoded values
encoder = OneHotEncoder(inputCols=["Weather_conditions_Index"], outputCols=["Weather_conditions_Encoded"])

# Fit and transform
encoded = encoder.fit(indexed).transform(indexed)

# Show the results
encoded.select("Weather_conditions", "Weather_conditions_Index", "Weather_conditions_Encoded").show()


23/12/09 02:32:12 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


+------------------+------------------------+--------------------------+
|Weather_conditions|Weather_conditions_Index|Weather_conditions_Encoded|
+------------------+------------------------+--------------------------+
|            Normal|                     0.0|             (8,[0],[1.0])|
|            Normal|                     0.0|             (8,[0],[1.0])|
|            Normal|                     0.0|             (8,[0],[1.0])|
|            Normal|                     0.0|             (8,[0],[1.0])|
|            Normal|                     0.0|             (8,[0],[1.0])|
|            Normal|                     0.0|             (8,[0],[1.0])|
|            Normal|                     0.0|             (8,[0],[1.0])|
|            Normal|                     0.0|             (8,[0],[1.0])|
|            Normal|                     0.0|             (8,[0],[1.0])|
|            Normal|                     0.0|             (8,[0],[1.0])|
|            Normal|                     0.0|      

# Cleaning Type_of_collision

In [31]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer, OneHotEncoder

# Initialize Spark session
spark = SparkSession.builder.appName("DataCleaning").getOrCreate()

# Read in the data  
df = spark.read.csv('Road.csv', header=True, inferSchema=True)

# Filter out rows with null or 'Unknown' values in 'Weather_conditions'
df_filtered = df.filter((df.Type_of_collision.isNotNull()) & (df.Type_of_collision != 'Unknown'))

# StringIndexer to convert the 'Type_of_collision' column to numeric indices
stringIndexer = StringIndexer(inputCol="Type_of_collision", outputCol="Type_of_collision_Index")

# Fit the indexer to the data and transform it
indexed = stringIndexer.fit(df).transform(df_filtered)

# OneHotEncoder to convert indexed numbers to one-hot encoded values
encoder = OneHotEncoder(inputCols=["Type_of_collision_Index"], outputCols=["Type_of_collision_Encoded"])

# Fit and transform
encoded = encoder.fit(indexed).transform(indexed)

# Show the results
encoded.select("Type_of_collision", "Type_of_collision_Index", "Type_of_collision_Encoded").show()


+--------------------+-----------------------+-------------------------+
|   Type_of_collision|Type_of_collision_Index|Type_of_collision_Encoded|
+--------------------+-----------------------+-------------------------+
|Collision with ro...|                    5.0|            (9,[5],[1.0])|
|Vehicle with vehi...|                    0.0|            (9,[0],[1.0])|
|Collision with ro...|                    1.0|            (9,[1],[1.0])|
|Vehicle with vehi...|                    0.0|            (9,[0],[1.0])|
|Vehicle with vehi...|                    0.0|            (9,[0],[1.0])|
|Vehicle with vehi...|                    0.0|            (9,[0],[1.0])|
|Vehicle with vehi...|                    0.0|            (9,[0],[1.0])|
|Vehicle with vehi...|                    0.0|            (9,[0],[1.0])|
|Collision with ro...|                    5.0|            (9,[5],[1.0])|
|Collision with ro...|                    5.0|            (9,[5],[1.0])|
|Collision with an...|                    4.0|     

# Cleaning Number_of_vehicles_involved

In [32]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import MinMaxScaler, VectorAssembler
from pyspark.ml.linalg import Vectors

# Initialize Spark session
spark = SparkSession.builder.appName("DataCleaning").getOrCreate()

# Read in the data  
df = spark.read.csv('Road.csv', header=True, inferSchema=True)

# Use VectorAssembler to create a vector column
assembler = VectorAssembler(inputCols=["Number_of_vehicles_involved"], outputCol="Number_of_vehicles_involved_vec")

# Transform the DataFrame
df_vector = assembler.transform(df)

# Initialize Min-Max Scaler
scaler = MinMaxScaler(inputCol="Number_of_vehicles_involved_vec", outputCol="Number_of_vehicles_involved_scaled")

# Fit and Transform the data
scalerModel = scaler.fit(df_vector)
scaledData = scalerModel.transform(df_vector)

# Show results
scaledData.select("Number_of_vehicles_involved", "Number_of_vehicles_involved_scaled").show()


+---------------------------+----------------------------------+
|Number_of_vehicles_involved|Number_of_vehicles_involved_scaled|
+---------------------------+----------------------------------+
|                          2|              [0.16666666666666...|
|                          2|              [0.16666666666666...|
|                          2|              [0.16666666666666...|
|                          2|              [0.16666666666666...|
|                          2|              [0.16666666666666...|
|                          1|                             [0.0]|
|                          1|                             [0.0]|
|                          2|              [0.16666666666666...|
|                          2|              [0.16666666666666...|
|                          2|              [0.16666666666666...|
|                          2|              [0.16666666666666...|
|                          2|              [0.16666666666666...|
|                        

# Cleaning Number_of_casualties

In [33]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import MinMaxScaler, VectorAssembler

# Initialize Spark session
spark = SparkSession.builder.appName("DataCleaning").getOrCreate()

# Read in the data  
df = spark.read.csv('Road.csv', header=True, inferSchema=True)

# Use VectorAssembler to create a vector column
assembler = VectorAssembler(inputCols=["Number_of_casualties"], outputCol="Number_of_casualties_vec")

# Transform the DataFrame
df_vector = assembler.transform(df)

# Initialize Min-Max Scaler
scaler = MinMaxScaler(inputCol="Number_of_casualties_vec", outputCol="Number_of_casualties_scaled")

# Fit and Transform the data
scalerModel = scaler.fit(df_vector)
scaledData = scalerModel.transform(df_vector)

# Show results
scaledData.select("Number_of_casualties", "Number_of_casualties_scaled").show()


+--------------------+---------------------------+
|Number_of_casualties|Number_of_casualties_scaled|
+--------------------+---------------------------+
|                   2|       [0.14285714285714...|
|                   2|       [0.14285714285714...|
|                   2|       [0.14285714285714...|
|                   2|       [0.14285714285714...|
|                   2|       [0.14285714285714...|
|                   1|                      [0.0]|
|                   1|                      [0.0]|
|                   1|                      [0.0]|
|                   1|                      [0.0]|
|                   1|                      [0.0]|
|                   1|                      [0.0]|
|                   1|                      [0.0]|
|                   2|       [0.14285714285714...|
|                   2|       [0.14285714285714...|
|                   2|       [0.14285714285714...|
|                   3|       [0.2857142857142857]|
|                   3|       [0

# Cleaning Vehicle_movement

In [34]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.builder.appName("example").getOrCreate()

# Read in the data  
df = spark.read.csv('Road.csv', header=True, inferSchema=True)

# Filter out rows with null or 'Unknown' values in 'Vehicle_movement'
df_filtered = df.filter((df.Vehicle_movement.isNotNull()) & (df.Vehicle_movement != 'Unknown'))

# StringIndexer to convert the 'Vehicle_movement' column to numeric indices
stringIndexer = StringIndexer(inputCol="Vehicle_movement", outputCol="Vehicle_movement_Index")

# Fit the indexer to the data and transform it
indexed = stringIndexer.fit(df).transform(df_filtered)

# OneHotEncoder to convert indexed numbers to one-hot encoded values
encoder = OneHotEncoder(inputCols=["Vehicle_movement_Index"], outputCols=["Vehicle_movement_Encoded"])

# Fit and transform
encoded = encoder.fit(indexed).transform(indexed)

# Show the results
encoded.select("Vehicle_movement", "Vehicle_movement_Index", "Vehicle_movement_Encoded").show()


23/12/09 02:32:46 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


+----------------+----------------------+------------------------+
|Vehicle_movement|Vehicle_movement_Index|Vehicle_movement_Encoded|
+----------------+----------------------+------------------------+
|  Going straight|                   0.0|          (12,[0],[1.0])|
|  Going straight|                   0.0|          (12,[0],[1.0])|
|  Going straight|                   0.0|          (12,[0],[1.0])|
|  Going straight|                   0.0|          (12,[0],[1.0])|
|  Going straight|                   0.0|          (12,[0],[1.0])|
|          U-Turn|                  10.0|         (12,[10],[1.0])|
| Moving Backward|                   1.0|          (12,[1],[1.0])|
|          U-Turn|                  10.0|         (12,[10],[1.0])|
|  Going straight|                   0.0|          (12,[0],[1.0])|
|          U-Turn|                  10.0|         (12,[10],[1.0])|
|        Turnover|                   4.0|          (12,[4],[1.0])|
|  Going straight|                   0.0|          (12,[0],[1.

# Cleaning Casualty_class

In [35]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.builder.appName("example").getOrCreate()

# Read in the data  
df = spark.read.csv('Road.csv', header=True, inferSchema=True)

# Filter out rows with null or 'Unknown' values in 'Casualty_class'
df_filtered = df.filter((df.Casualty_class.isNotNull()) & (df.Casualty_class != 'na'))

# StringIndexer to convert the 'Casualty_class' column to numeric indices
stringIndexer = StringIndexer(inputCol="Casualty_class", outputCol="Casualty_class_Index")

# Fit the indexer to the data and transform it
indexed = stringIndexer.fit(df).transform(df_filtered)

# OneHotEncoder to convert indexed numbers to one-hot encoded values
encoder = OneHotEncoder(inputCols=["Casualty_class_Index"], outputCols=["Casualty_class_Encoded"])

# Fit and transform
encoded = encoder.fit(indexed).transform(indexed)

# Show the results
encoded.select("Casualty_class", "Casualty_class_Index", "Casualty_class_Encoded").show()


+---------------+--------------------+----------------------+
| Casualty_class|Casualty_class_Index|Casualty_class_Encoded|
+---------------+--------------------+----------------------+
|Driver or rider|                 0.0|         (3,[0],[1.0])|
|     Pedestrian|                 2.0|         (3,[2],[1.0])|
|Driver or rider|                 0.0|         (3,[0],[1.0])|
|Driver or rider|                 0.0|         (3,[0],[1.0])|
|     Pedestrian|                 2.0|         (3,[2],[1.0])|
|      Passenger|                 3.0|             (3,[],[])|
|Driver or rider|                 0.0|         (3,[0],[1.0])|
|Driver or rider|                 0.0|         (3,[0],[1.0])|
|Driver or rider|                 0.0|         (3,[0],[1.0])|
|Driver or rider|                 0.0|         (3,[0],[1.0])|
|      Passenger|                 3.0|             (3,[],[])|
|      Passenger|                 3.0|             (3,[],[])|
|Driver or rider|                 0.0|         (3,[0],[1.0])|
|     Pe

# Cleaning Sex_of_casualty

In [47]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.builder.appName("example").getOrCreate()

# Read in the data  
df = spark.read.csv('Road.csv', header=True, inferSchema=True)

# Filter out rows with null or 'Unknown' values in 'Sex_of_casualty'
df_filtered = df.filter((df.Sex_of_casualty.isNotNull()) & (df.Sex_of_casualty != 'na'))

# StringIndexer to convert the 'Sex_of_casualty' column to numeric indices
stringIndexer = StringIndexer(inputCol="Sex_of_casualty", outputCol="Sex_of_casualty_Index")

# Fit the indexer to the data and transform it
indexed = stringIndexer.fit(df).transform(df_filtered)

# OneHotEncoder to convert indexed numbers to one-hot encoded values
encoder = OneHotEncoder(inputCols=["Sex_of_casualty_Index"], outputCols=["Sex_of_casualty_Encoded"])

# Fit and transform
encoded = encoder.fit(indexed).transform(indexed)

# Show the results
encoded.select("Sex_of_casualty", "Sex_of_casualty_Index", "Sex_of_casualty_Encoded").show()


+---------------+---------------------+-----------------------+
|Sex_of_casualty|Sex_of_casualty_Index|Sex_of_casualty_Encoded|
+---------------+---------------------+-----------------------+
|           Male|                  0.0|          (2,[0],[1.0])|
|         Female|                  2.0|              (2,[],[])|
|           Male|                  0.0|          (2,[0],[1.0])|
|         Female|                  2.0|              (2,[],[])|
|           Male|                  0.0|          (2,[0],[1.0])|
|           Male|                  0.0|          (2,[0],[1.0])|
|           Male|                  0.0|          (2,[0],[1.0])|
|         Female|                  2.0|              (2,[],[])|
|           Male|                  0.0|          (2,[0],[1.0])|
|           Male|                  0.0|          (2,[0],[1.0])|
|         Female|                  2.0|              (2,[],[])|
|           Male|                  0.0|          (2,[0],[1.0])|
|         Female|                  2.0| 

# Cleaning Age_band_of_casualty

In [37]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.builder.appName("example").getOrCreate()

# Read in the data  
df = spark.read.csv('Road.csv', header=True, inferSchema=True)

# Filter out rows with null or 'Unknown' values in 'Age_band_of_casualty'
df_filtered = df.filter((df.Age_band_of_casualty.isNotNull()) & (df.Age_band_of_casualty != 'na'))

# StringIndexer to convert the 'Age_band_of_casualty' column to numeric indices
stringIndexer = StringIndexer(inputCol="Age_band_of_casualty", outputCol="Age_band_of_casualty_Index")

# Fit the indexer to the data and transform it
indexed = stringIndexer.fit(df).transform(df_filtered)

# OneHotEncoder to convert indexed numbers to one-hot encoded values
encoder = OneHotEncoder(inputCols=["Age_band_of_casualty_Index"], outputCols=["Age_band_of_casualty_Encoded"])

# Fit and transform
encoded = encoder.fit(indexed).transform(indexed)

# Show the results
encoded.select("Age_band_of_casualty", "Age_band_of_casualty_Index", "Age_band_of_casualty_Encoded").show()


+--------------------+--------------------------+----------------------------+
|Age_band_of_casualty|Age_band_of_casualty_Index|Age_band_of_casualty_Encoded|
+--------------------+--------------------------+----------------------------+
|               31-50|                       2.0|               (5,[2],[1.0])|
|               18-30|                       1.0|               (5,[1],[1.0])|
|               31-50|                       2.0|               (5,[2],[1.0])|
|               18-30|                       1.0|               (5,[1],[1.0])|
|            Under 18|                       3.0|               (5,[3],[1.0])|
|               18-30|                       1.0|               (5,[1],[1.0])|
|               18-30|                       1.0|               (5,[1],[1.0])|
|               18-30|                       1.0|               (5,[1],[1.0])|
|            Under 18|                       3.0|               (5,[3],[1.0])|
|               18-30|                       1.0|   

# Cleaning Casualty_severity

In [38]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.builder.appName("example").getOrCreate()

# Read in the data  
df = spark.read.csv('Road.csv', header=True, inferSchema=True)

# Filter out rows with null or 'Unknown' values in 'Casualty_severity'
df_filtered = df.filter((df.Casualty_severity.isNotNull()) & (df.Casualty_severity != 'na'))

# StringIndexer to convert the 'Casualty_severity' column to numeric indices
stringIndexer = StringIndexer(inputCol="Casualty_severity", outputCol="Casualty_severity_Index")

# Fit the indexer to the data and transform it
indexed = stringIndexer.fit(df).transform(df_filtered)

# OneHotEncoder to convert indexed numbers to one-hot encoded values
encoder = OneHotEncoder(inputCols=["Casualty_severity_Index"], outputCols=["Casualty_severity_Encoded"])

# Fit and transform
encoded = encoder.fit(indexed).transform(indexed)

# Show the results
encoded.select("Casualty_severity", "Casualty_severity_Index", "Casualty_severity_Encoded").show()


+-----------------+-----------------------+-------------------------+
|Casualty_severity|Casualty_severity_Index|Casualty_severity_Encoded|
+-----------------+-----------------------+-------------------------+
|                3|                    0.0|            (3,[0],[1.0])|
|                3|                    0.0|            (3,[0],[1.0])|
|                3|                    0.0|            (3,[0],[1.0])|
|                3|                    0.0|            (3,[0],[1.0])|
|                3|                    0.0|            (3,[0],[1.0])|
|                3|                    0.0|            (3,[0],[1.0])|
|                3|                    0.0|            (3,[0],[1.0])|
|                3|                    0.0|            (3,[0],[1.0])|
|                3|                    0.0|            (3,[0],[1.0])|
|                3|                    0.0|            (3,[0],[1.0])|
|                3|                    0.0|            (3,[0],[1.0])|
|                3| 

# Cleaning Work_of_casuality

In [39]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.builder.appName("example").getOrCreate()

# Read in the data  
df = spark.read.csv('Road.csv', header=True, inferSchema=True)

# Filter out rows with null or 'Unknown' values in 'Work_of_casuality'
df_filtered = df.filter((df.Work_of_casuality.isNotNull()) & (df.Work_of_casuality != 'Unknown'))

# StringIndexer to convert the 'Work_of_casuality' column to numeric indices
stringIndexer = StringIndexer(inputCol="Work_of_casuality", outputCol="Work_of_casuality_Index")

# Fit the indexer to the data and transform it
indexed = stringIndexer.fit(df).transform(df_filtered)

# OneHotEncoder to convert indexed numbers to one-hot encoded values
encoder = OneHotEncoder(inputCols=["Work_of_casuality_Index"], outputCols=["Work_of_casuality_Encoded"])

# Fit and transform
encoded = encoder.fit(indexed).transform(indexed)

# Show the results
encoded.select("Work_of_casuality", "Work_of_casuality_Index", "Work_of_casuality_Encoded").show()


+-----------------+-----------------------+-------------------------+
|Work_of_casuality|Work_of_casuality_Index|Work_of_casuality_Encoded|
+-----------------+-----------------------+-------------------------+
|           Driver|                    0.0|            (6,[0],[1.0])|
|           Driver|                    0.0|            (6,[0],[1.0])|
|           Driver|                    0.0|            (6,[0],[1.0])|
|           Driver|                    0.0|            (6,[0],[1.0])|
|           Driver|                    0.0|            (6,[0],[1.0])|
|           Driver|                    0.0|            (6,[0],[1.0])|
|           Driver|                    0.0|            (6,[0],[1.0])|
|           Driver|                    0.0|            (6,[0],[1.0])|
|            Other|                    3.0|            (6,[3],[1.0])|
|           Driver|                    0.0|            (6,[0],[1.0])|
|           Driver|                    0.0|            (6,[0],[1.0])|
|           Driver| 

# Cleaning Fitness_of_casuality

In [45]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.sql.functions import when

# Initialize Spark session
spark = SparkSession.builder.appName("example").getOrCreate()

# Read in the data  
df = spark.read.csv('Road.csv', header=True, inferSchema=True)

# Filter out rows with null or 'Unknown' values in 'Fitness_of_casuality'
df_filtered = df.filter((df.Fitness_of_casuality.isNotNull()) & (df.Fitness_of_casuality != 'Unknown'))

# Replace 'NormalNormal' with 'Normal' in 'Fitness_of_casuality'
df_filtered = df_filtered.withColumn("Fitness_of_casuality", 
                   when(df.Fitness_of_casuality == "NormalNormal", "Normal")
                   .otherwise(df.Fitness_of_casuality))

# StringIndexer to convert the 'Fitness_of_casuality' column to numeric indices
stringIndexer = StringIndexer(inputCol="Fitness_of_casuality", outputCol="Fitness_of_casuality_Index")

# Fit the indexer to the data and transform it
indexed = stringIndexer.fit(df_filtered).transform(df_filtered)

# OneHotEncoder to convert indexed numbers to one-hot encoded values
# encoder = OneHotEncoder(inputCols=["Fitness_of_casuality_Index"], outputCols=["Fitness_of_casuality_Encoded"], dropLast=False)
encoder = OneHotEncoder(inputCols=["Fitness_of_casuality_Index"], outputCols=["Fitness_of_casuality_Encoded"])

# Fit and transform
encoded = encoder.fit(indexed).transform(indexed)

# Show the results
encoded.select("Fitness_of_casuality", "Fitness_of_casuality_Index", "Fitness_of_casuality_Encoded").show()


+--------------------+--------------------------+----------------------------+
|Fitness_of_casuality|Fitness_of_casuality_Index|Fitness_of_casuality_Encoded|
+--------------------+--------------------------+----------------------------+
|              Normal|                       0.0|               (3,[0],[1.0])|
|              Normal|                       0.0|               (3,[0],[1.0])|
|              Normal|                       0.0|               (3,[0],[1.0])|
|              Normal|                       0.0|               (3,[0],[1.0])|
|              Normal|                       0.0|               (3,[0],[1.0])|
|              Normal|                       0.0|               (3,[0],[1.0])|
|              Normal|                       0.0|               (3,[0],[1.0])|
|              Normal|                       0.0|               (3,[0],[1.0])|
|              Normal|                       0.0|               (3,[0],[1.0])|
|              Normal|                       0.0|   

# Cleaning Pedestrian_movement

In [48]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.sql.functions import when

# Initialize Spark session
spark = SparkSession.builder.appName("example").getOrCreate()

# Read in the data  
df = spark.read.csv('Road.csv', header=True, inferSchema=True)

# Filter out rows with null or 'Unknown' values in 'Pedestrian_movement'
df_filtered = df.filter((df.Pedestrian_movement.isNotNull()) & (df.Pedestrian_movement != 'Unknown or other'))

# StringIndexer to convert the 'Pedestrian_movement' column to numeric indices
stringIndexer = StringIndexer(inputCol="Pedestrian_movement", outputCol="Pedestrian_movement_Index")

# Fit the indexer to the data and transform it
indexed = stringIndexer.fit(df_filtered).transform(df_filtered)

# OneHotEncoder to convert indexed numbers to one-hot encoded values
encoder = OneHotEncoder(inputCols=["Pedestrian_movement_Index"], outputCols=["Pedestrian_movement_Encoded"])

# Fit and transform
encoded = encoder.fit(indexed).transform(indexed)

# Show the results
encoded.select("Pedestrian_movement", "Pedestrian_movement_Index", "Pedestrian_movement_Encoded").show()


+--------------------+-------------------------+---------------------------+
| Pedestrian_movement|Pedestrian_movement_Index|Pedestrian_movement_Encoded|
+--------------------+-------------------------+---------------------------+
|    Not a Pedestrian|                      0.0|              (7,[0],[1.0])|
|    Not a Pedestrian|                      0.0|              (7,[0],[1.0])|
|    Not a Pedestrian|                      0.0|              (7,[0],[1.0])|
|    Not a Pedestrian|                      0.0|              (7,[0],[1.0])|
|    Not a Pedestrian|                      0.0|              (7,[0],[1.0])|
|    Not a Pedestrian|                      0.0|              (7,[0],[1.0])|
|    Not a Pedestrian|                      0.0|              (7,[0],[1.0])|
|    Not a Pedestrian|                      0.0|              (7,[0],[1.0])|
|Crossing from dri...|                      2.0|              (7,[2],[1.0])|
|    Not a Pedestrian|                      0.0|              (7,[0],[1.0])|

# Cleaning Cause_of_accident

In [49]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.sql.functions import when

# Initialize Spark session
spark = SparkSession.builder.appName("example").getOrCreate()

# Read in the data  
df = spark.read.csv('Road.csv', header=True, inferSchema=True)

# Filter out rows with null or 'Unknown' values in 'Cause_of_accident'
df_filtered = df.filter((df.Cause_of_accident.isNotNull()) & (df.Cause_of_accident != 'Unknown') & (df.Cause_of_accident != 'Other'))

# StringIndexer to convert the 'Cause_of_accident' column to numeric indices
stringIndexer = StringIndexer(inputCol="Cause_of_accident", outputCol="Cause_of_accident_Index")

# Fit the indexer to the data and transform it
indexed = stringIndexer.fit(df_filtered).transform(df_filtered)

# OneHotEncoder to convert indexed numbers to one-hot encoded values
encoder = OneHotEncoder(inputCols=["Cause_of_accident_Index"], outputCols=["Cause_of_accident_Encoded"])

# Fit and transform
encoded = encoder.fit(indexed).transform(indexed)

# Show the results
encoded.select("Cause_of_accident", "Cause_of_accident_Index", "Cause_of_accident_Encoded").show()


+--------------------+-----------------------+-------------------------+
|   Cause_of_accident|Cause_of_accident_Index|Cause_of_accident_Encoded|
+--------------------+-----------------------+-------------------------+
|     Moving Backward|                    5.0|           (17,[5],[1.0])|
|          Overtaking|                    7.0|           (17,[7],[1.0])|
|Changing lane to ...|                    2.0|           (17,[2],[1.0])|
|Changing lane to ...|                    1.0|           (17,[1],[1.0])|
|          Overtaking|                    7.0|           (17,[7],[1.0])|
|         Overloading|                   15.0|          (17,[15],[1.0])|
|No priority to ve...|                    4.0|           (17,[4],[1.0])|
|Changing lane to ...|                    1.0|           (17,[1],[1.0])|
|     Moving Backward|                    5.0|           (17,[5],[1.0])|
|Changing lane to ...|                    2.0|           (17,[2],[1.0])|
|No priority to pe...|                    6.0|     

# Cleaning Accident_severity

In [50]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.sql.functions import when

# Initialize Spark session
spark = SparkSession.builder.appName("example").getOrCreate()

# Read in the data  
df = spark.read.csv('Road.csv', header=True, inferSchema=True)

# Filter out rows with null or 'Unknown' values in 'Accident_severity'
df_filtered = df.filter((df.Accident_severity.isNotNull()) & (df.Accident_severity != 'Unknown'))

# StringIndexer to convert the 'Accident_severity' column to numeric indices
stringIndexer = StringIndexer(inputCol="Accident_severity", outputCol="Accident_severity_Index")

# Fit the indexer to the data and transform it
indexed = stringIndexer.fit(df_filtered).transform(df_filtered)

# OneHotEncoder to convert indexed numbers to one-hot encoded values
encoder = OneHotEncoder(inputCols=["Accident_severity_Index"], outputCols=["Accident_severity_Encoded"])

# Fit and transform
encoded = encoder.fit(indexed).transform(indexed)

# Show the results
encoded.select("Accident_severity", "Accident_severity_Index", "Accident_severity_Encoded").show()


+-----------------+-----------------------+-------------------------+
|Accident_severity|Accident_severity_Index|Accident_severity_Encoded|
+-----------------+-----------------------+-------------------------+
|    Slight Injury|                    0.0|            (2,[0],[1.0])|
|    Slight Injury|                    0.0|            (2,[0],[1.0])|
|   Serious Injury|                    1.0|            (2,[1],[1.0])|
|    Slight Injury|                    0.0|            (2,[0],[1.0])|
|    Slight Injury|                    0.0|            (2,[0],[1.0])|
|    Slight Injury|                    0.0|            (2,[0],[1.0])|
|    Slight Injury|                    0.0|            (2,[0],[1.0])|
|    Slight Injury|                    0.0|            (2,[0],[1.0])|
|    Slight Injury|                    0.0|            (2,[0],[1.0])|
|   Serious Injury|                    1.0|            (2,[1],[1.0])|
|   Serious Injury|                    1.0|            (2,[1],[1.0])|
|   Serious Injury| 

In [ ]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.sql import SparkSession
from pyspark.sql.functions import monotonically_increasing_id

# Add an index column to the original DataFrame
df = df.withColumn("index", monotonically_increasing_id())

# List of columns to be encoded
columns_to_encode = [
    #"Age_band_of_driver", "Sex_of_driver", "Educational_level", 
    #"Vehicle_driver_relation", "Driving_experience", "Type_of_vehicle", 
    "Owner_of_vehicle", "Service_year_of_vehicle", "Defect_of_vehicle", 
    #"Area_accident_occured", "Lanes_or_Medians", "Road_allignment", 
    #"Types_of_Junction", "Road_surface_type", "Road_surface_conditions", 
    #"Light_conditions"
]

for col in columns_to_encode:
    # Filter out rows with null or 'Unknown' values in the current column
    df_filtered = df.filter((df[col].isNotNull()) & (df[col] != 'Unknown'))

    # StringIndexer to convert the current column to numeric indices, handling unseen labels
    stringIndexer = StringIndexer(inputCol=col, outputCol=f"{col}_Index", handleInvalid="keep")

    # Fit the indexer to the data and transform it
    indexed = stringIndexer.fit(df_filtered).transform(df_filtered)

    # OneHotEncoder to convert indexed numbers to one-hot encoded values
    encoder = OneHotEncoder(inputCols=[f"{col}_Index"], outputCols=[f"{col}_Encoded"])

    # Check if the encoded column already exists
    if f"{col}_Encoded" in indexed.columns:
        indexed = indexed.drop(f"{col}_Encoded")

    # Fit and transform
    encoded = encoder.fit(indexed).transform(indexed)

    # Add the same index column to the encoded DataFrame
    encoded = encoded.withColumn("index", monotonically_increasing_id())

    # Join using the index
    df = df.join(encoded.select("index", f"{col}_Encoded"), on=["index"])

# Optionally, you can drop the index column after joining if it's no longer needed
df = df.drop("index")

# Show the results
df.show()